based on https://www.kaggle.com/competitions/word2vec-nlp-tutorial/overview

In [6]:
!cp kaggle.json /content
!chmod 600 /content/kaggle.json

In [6]:
import numpy as np
import pandas as pd 

In [ ]:
!pip install kaggle

In [2]:

## set up custom path to kaggle.json: 
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

from pathlib import Path
path = Path('word2vec-nlp-tutorial')
if not path.exists():
    import zipfile,kaggle
    kaggle.api.competition_download_cli(str(path))
    # for each zip file in the path unzip it
    for f in path.glob('*.zip'):
        zipfile.ZipFile(f).extractall(path)

In [7]:
train =  pd.read_csv(path/'labeledTrainData.tsv',sep='\t')

In [8]:
train.describe(include='object')

,id,review
count,25000,25000
unique,25000,24904
top,5814_8,"When i got this movie free from my job, along ..."
freq,1,3


In [10]:
train.review[0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [11]:
from bs4 import BeautifulSoup

In [12]:
bs1 = BeautifulSoup(train.review[0])
bs1.get_text()

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 20 mi